## <font color = 'darkred'> **Contributions of Muons from Pions and Kaons** </font>

### <font color = 'darkgreen'> **Packages** </font>

In [ ]:
from MCEq.core import config, MCEqRun
import crflux.models as crf
import matplotlib.pyplot as plt
import scipy
import numpy as np
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages

### Functions

In [ ]:
# Function for the fraction of muons from sources
def frac_in_mu(energy:int, particle: str, month: str, interaction = 'SIBYLL23C'):
    mceq = MCEqRun(
    interaction_model = interaction, # High-energy hadronic interaction model
    primary_model = (crf.HillasGaisser2012, 'H3a'),   # cosmic ray flux at the top of the atmosphere
    theta_deg = 0.  # zenith angle - 0 indicates vertical airshower
    )

    if particle == 'p':
        mceq.set_single_primary_particle(energy, pdg_id=2212) # energies are in units of GeV

    elif particle == 'fe':
        mceq.set_single_primary_particle(energy, corsika_id = 5626) # energies are in units of GeV
    
    h_grid = np.linspace(50 * 1e3 * 1e2, 283500) # altitudes up until observation level of IceCube (in cm)
    X_grid = mceq.density_model.h2X(h_grid)
    mceq.set_density_model(('MSIS00_IC', ('SouthPole', month)))
    
    mceq.solve(int_grid = X_grid)
    
    mu_total = mceq.get_solution('total_mu+') + mceq.get_solution('total_mu-')
    mu_pi = mceq.get_solution('pi_mu+') + mceq.get_solution('pi_mu-')
    mu_k = mceq.get_solution('k_mu+') + mceq.get_solution('k_mu-')
    mu_other = mu_total - (mu_pi + mu_k) # contribution of muons from other sources
    mu_prompt  = mceq.get_solution('pr_mu+') + mceq.get_solution('pr_mu-')
    E = mceq.e_grid

    return E, mu_total, mu_pi, mu_k, mu_other, mu_prompt


# Function for number of muons from sources

def tot_muons_frac(energy:int, particle: str, month: str, interaction = 'SIBYLL23C'):
    mceq = MCEqRun(
    interaction_model = interaction, # High-energy hadronic interaction model
    primary_model = (crf.HillasGaisser2012, 'H3a'),   # cosmic ray flux at the top of the atmosphere
    theta_deg = 0.  # zenith angle - 0 indicates vertical airshower
    )

    if particle == 'p':
        mceq.set_single_primary_particle(energy, pdg_id=2212) # energies are in units of GeV

    elif particle == 'fe':
        mceq.set_single_primary_particle(energy, corsika_id = 5626) # energies are in units of GeV
    
    h_grid = np.linspace(50 * 1e3 * 1e2, 283500) # altitudes up until observation level of IceCube (in cm)
    X_grid = mceq.density_model.h2X(h_grid)
    mceq.set_density_model(('MSIS00_IC', ('SouthPole', month)))
    
    mceq.solve(int_grid = X_grid)
    
    mu_total_arr = mceq.get_solution('total_mu+') + mceq.get_solution('total_mu-')
    mu_pi_arr = mceq.get_solution('pi_mu+') + mceq.get_solution('pi_mu-')
    mu_k_arr = mceq.get_solution('k_mu+') + mceq.get_solution('k_mu-')
    mu_prompt = mceq.get_solution('pr_mu+') + mceq.get_solution('pr_mu-')

    E = mceq.e_grid

    # following code from Stef
    
    spline_tot = scipy.interpolate.CubicSpline(E, mu_total_arr)
    spline_pi = scipy.interpolate.CubicSpline(E, mu_pi_arr)
    spline_k = scipy.interpolate.CubicSpline(E, mu_k_arr)
    spline_other = scipy.interpolate.CubicSpline(E, mu_prompt)

    # The following code will return the value AND the uncertainty -- since we only want the value, I will only take the first index
    tot_mu = scipy.integrate.quad(lambda x : spline_tot(x), 500, 1e5)
    tot_pi_mu = scipy.integrate.quad(lambda x : spline_pi(x), 500, 1e5)
    tot_k_mu = scipy.integrate.quad(lambda x : spline_k(x), 500, 1e5)
    tot_mu_other = scipy.integrate.quad(lambda x : spline_other(x), 500, 1e5)

    #tot_mu_other =  tot_mu[0] - (tot_pi_mu[0] + tot_k_mu[0])
    
    return tot_mu[0], tot_pi_mu[0], tot_k_mu[0], tot_mu_other[0]

#### <font color = 'indigo'> **Task**</font>
> Obtain the spectra of muons coming from pions, kaons, and other particles. Put all of them on one plot together with the total spectrum. 
> * This notebook should be very helpful: https://github.com/mceq-project/MCEq/blob/master/examples/Partial_hadron_contribution.ipynb
> 
> 
> Start with a single cosmic-ray energy, mass, and atmosphere. Once we have that one looking good, the way you set things up it will hopefully be easy again to produce them for other combinations.

#### <font color = 'black'>**Questions:**</font>

<font color = 'darkgreen'> **Why do we call the groups in MCEq's "resonances"?** </font>
* Refers to specific energy levels within the particle interactions that occur within the shower. In this case, certain particles are more likely to be produced due to interaction with atmospheric nuclei. (Double-check)

<font color = 'darkgreen'> **What is "mag" in the reference Python notebook?** </font>
* *mag* stands for the magnification factor. It is an opitional argument and if we set this value to a number, the solution is multiplied by the magnetification factor (Flux*Magnification Factor)

<font color = 'darkgreen'> **What is the difference between *conv*, *prompt*, and *total* ?** </font>
* The **total** flux is the combined flux of muons (eg. muons and muon neutrinos) from **ALL sources**.
* The **conventional** flux is the flux of muons from **ONLY pions and kaons**.
* The **prompt** is the flux of muons that come from **other sources** (not pions or kaons).

<font color = 'darkgreen'> **What is the difference between mu_pi and mu_k?** </font>
* *mu_pi* is the flux of muons that are directly contributed from pions and *mu_k* is the flux of muons directly contributed from kaons.

In [ ]:
prime_ener = [10, 100]
for energy in prime_ener:
    E, tot_mu, mu_pi, mu_k, mu_other = np.loadtxt('Muon_Sources\{}\{}PeV\Average_{}PeV.txt'.format('Fe', energy, energy), unpack=True)
    fig, ax = plt.subplots(2, 1, sharex = True, gridspec_kw = {'height_ratios': (8,6) , 'hspace': 0.12})
    ax[0].loglog(E, tot_mu, label = 'Total', color = 'darkorange')
    ax[0].loglog(E, mu_pi, label = 'From Pions', color = 'darkgreen')
    ax[0].loglog(E, mu_k, label = 'From Kaons', color = 'indigo')
    ax[0].set_xlim(0.05, 2e5)
    ax[0].set_ylim(0.01, 1e6)
    ax[0].set_ylabel(r'$dN_\mu / dE_\mu \; (GeV^{-1})$', fontsize = 11)
    #ax.set_xlabel('Energy (GeV)')
    ax[0].legend(fontsize = 10)

    other = tot_mu - (mu_pi + mu_k)
    ax[1].loglog(E, mu_pi / tot_mu, label = 'From Pions', color = 'darkgreen')
    ax[1].loglog(E, mu_k / tot_mu, label = 'From Kaons', color = 'indigo')
    ax[1].loglog(E, other/tot_mu, label = 'Other', color = 'mediumvioletred')
    ax[1].axhline(1, color = 'grey', ls = '--')
    ax[1].set_xlim(0.05, 1e5)
    ax[1].set_ylim(0.001, 3)
    ax[1].set_ylabel('Fraction of Muons')
    ax[1].set_xlabel(r'$E_\mu \;(GeV)$', fontsize = 11)
    ax[1].legend(fontsize = 10)
    

    ax[0].set_title('Yearly Average of {} PeV Iron'.format(energy))

From the graph above, it seems that the contributions of muons from pions is much greater than the contribution from kaons. Although, it seems that for high energies, the amount of kaons that contribute to the total amount of muons increases.
* Recall that both pions and kaons are mesons, which means that they are composed of an **equal** number of quarks and antiquarks and are bounded by the strong interaction.  


One reason that the contributions from pions is dominant in the graph above is because pions primarily decay into a muon (and muon neutrino) during an air shower. However, kaons can also decay into a muon, pion, and muon neutrino, however, it primarily decays into a pion, which will decay to a muon. This means that regardless of the path, there are more pions that are created that decay into muons, causing the ratio to be closer greater compared to the kaon ratio.


I also noticed that the contribution of muons from kaons increases as the energy increases.
* Therefore it is likely that kaons are more likely to decay into muons at high energies.

![image.png](contributions_of_muons.png)

### Contributions of Pions and Kaons for Various Months

In [ ]:
Months = ['January', 'April', 'July', 'October', 'December']
prime_ener = [1e7, 10e7, 100e7, 1000e7]

In [ ]:
for month in Months:
    for energy in prime_ener:
        E, tot_mu, mu_pi, mu_k = frac_in_mu(energy, 'p', month)
        other = tot_mu - (mu_pi + mu_k)

        fig, ax = plt.subplots(2, 1, sharex = True, gridspec_kw = {'height_ratios': (8,6) , 'hspace': 0.12})
        ax[0].loglog(E, tot_mu, label = '$\mu_{total}$', color = 'darkorange')
        ax[0].loglog(E, mu_pi, label = '$\pi$', color = 'darkgreen')
        ax[0].loglog(E, mu_k, label = '$K$', color = 'indigo')
        ax[0].plot(E, other, label = 'Other', color = 'mediumvioletred')

        ax[0].set_xlim(0.05, 2e5)
        ax[0].set_ylim(0.01, 5e7)
        ax[0].set_ylabel(r'$dN_\mu / dE_\mu \; (GeV^{-1})$', fontsize = 11)
        ax[0].legend(loc='upper center', ncol = 4, bbox_to_anchor=(0.5, 1.35))
        ax[0].set_title('100e7 GeV Proton in April')

        ax[1].plot(E, mu_pi / tot_mu, label = 'From Pions', color = 'darkgreen')
        ax[1].plot(E, mu_k / tot_mu, label = 'From Kaons', color = 'indigo')
        ax[1].plot(E, other/tot_mu, label = 'Other', color = 'mediumvioletred')
        ax[1].axhline(1, color = 'grey', ls = '--')

        ax[1].set_xscale("log")
        ax[1].set_xlim(0.05, 1e5)
        ax[1].set_ylim(0.001, 1.1)
        ax[1].set_ylabel('Fraction of Muons')
        ax[1].set_xlabel(r'$E_\mu \;(GeV)$', fontsize = 11)
        
        if energy == 1e7:
            ax[0].set_title('1 PeV Proton in {}'.format(month))
            fig.savefig('1PeV_{}.png'.format(month))
        elif energy == 10e7:
            ax[0].set_title('10 PeV Proton in {}'.format(month))
            fig.savefig('10PeV_{}.png'.format(month))
        elif energy == 100e7:
            ax[0].set_title('100 PeV Proton in {}'.format(month))
            fig.savefig('100PeV_{}.png'.format(month))
        elif energy == 1000e7:
            ax[0].set_title('1000 PeV Proton in {}'.format(month))
            fig.savefig('1000PeV_{}.png'.format(month))